In [1]:
import yaml

config_path = '/workspaces/legal-advisor-rag/config.yaml'

with open(config_path, 'r') as file:
    config = yaml.safe_load(file)

In [2]:
import os
from dotenv import load_dotenv
from openai import OpenAI
from anthropic import Anthropic

load_dotenv(dotenv_path='/workspaces/legal-advisor-rag/legal-advisor-rag/.env')

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")

if not OPENAI_API_KEY:
    raise ValueError("Missing OpenAI API key.")
if not ANTHROPIC_API_KEY:
    raise ValueError("Missing Anthropic API key.")

openai_clent = OpenAI(api_key = OPENAI_API_KEY)
anthropic_client = Anthropic(api_key=ANTHROPIC_API_KEY)

llm_client_map = {
    "gpt-4o-mini": openai_clent,
    "claude-3-sonnet-20240229": anthropic_client
}

def get_llm_client(model_name):
    """
    Get the appropriate LLM client based on the model name.
    Raises an error if the model is not recognized.
    """
    llm_class = llm_client_map.get(model_name)
    if llm_class is None:
        raise ValueError(f"LLM client error: '{model_name}' is not recognized.")
    return llm_class

# Assuming config['RAG_model']['model_name'] is defined somewhere
model_name = config['RAG_model']['model_name']
llm_client = get_llm_client(model_name)

print(f"Successfully initialized {model_name} LLM client.")

Successfully initialized gpt-4o-mini LLM client.


In [3]:
Org =  config['Weaviate']['ORG']
API_KEY = config['Weaviate']['API_KEY']
weaviate_url =  config['Weaviate']['URL']

chunk_size = config['Create_node']['chunk_size']
chunk_overlap = config['Create_node']['chunk_overlap']

documents_file_path = config['Documents_path']

language = config['Language']

In [4]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings

# Use a more suitable model for sentence embeddings
embed_model_name = config.get('Embedding_model', 'sentence-transformers/distiluse-base-multilingual-cased-v2')

# Check if the model is a sentence-transformer model; if not, default to one that is
if "sentence-transformers" not in embed_model_name:
    print(f"Warning: The model {embed_model_name} is not a sentence-transformer model. Switching to a default.")
    embed_model_name = 'sentence-transformers/distiluse-base-multilingual-cased-v2'

embed_model = HuggingFaceEmbedding(model_name=embed_model_name)

# Apply the embedding model to settings
Settings.embed_model = embed_model

print(f"Using embedding model: {embed_model_name}")


Using embedding model: sentence-transformers/distiluse-base-multilingual-cased-v2


/home/codespace/.local/share/virtualenvs/legal-advisor-rag-x6CAXa7x/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [43]:
# tx = 'how are you?'
# embed_model.get_text_embedding(tx) == embed_model.get_query_embedding(tx)

# emb1 = embed_model.get_text_embedding(tx)
# emb2 = embed_model.get_query_embedding(tx)

# embed_model.similarity(embedding1=emb1, embedding2=emb2)

1.0000000000000002

# Ingestion

In [ ]:
# import weaviate

# def get_weaviate_client(api_key, url):
#     return weaviate.Client(
#             url=weaviate_url,
#             auth_client_secret=weaviate.AuthApiKey(api_key=api_key)
#         )

# # client
# client = get_weaviate_client(API_KEY, weaviate_url)

# print(f"Client is ready: {client.is_ready()}")

In [ ]:
# # DELETING all info from DB

# client.schema.delete_class(Org)

In [56]:
# client = weaviate.connect_to_wcs(
#             cluster_url=weaviate_url,
#             auth_credentials=weaviate.AuthApiKey(api_key=API_KEY)
# )

# print(client.is_ready())

True


In [85]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

/home/codespace/.local/share/virtualenvs/legal-advisor-rag-x6CAXa7x/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [96]:
# import tiktoken
# from llama_index.core import Document

# # Определяем кодировщик токенов для GPT-4
# encoding = tiktoken.encoding_for_model('gpt-4')



# def load_documents(file_path):
#     return SimpleDirectoryReader(file_path).load_data()


# # Подсчет токенов
# def count_tokens(text):
#     tokens = encoding.encode(text)
#     return len(tokens)

# # Функция для разбиения текста по токенам
# def split_text_by_tokens(documents, max_tokens=8192):
#     split_docs = []
    
#     for doc in documents:
#         law_text = doc.text
#         paragraphs = law_text.split('\n\n')  # Можно использовать абзацы для разбиения

#         current_text = ""
#         for paragraph in paragraphs:
#             if count_tokens(current_text + paragraph) < max_tokens:
#                 current_text += paragraph + "\n\n"
#             else:
#                 # Если достигли лимита, сохраняем текущий текст как отдельный документ
#                 split_docs.append(Document(text=current_text, metadata=doc.metadata))
#                 current_text = paragraph + "\n\n"
        
#         # Добавляем последний кусок текста
#         if current_text:
#             split_docs.append(Document(text=current_text, metadata=doc.metadata))

#     return split_docs




Original documents: 11, Split documents: 22


In [113]:
from llama_index.core import SimpleDirectoryReader, Document
import re


def load_documents(file_path):
    return SimpleDirectoryReader(file_path).load_data()

def count_tokens(text):
    tokens = tokenizer.tokenize(text)
    return len(tokens)

# Разбиение текста на параграфы, не превышающие 8000 токенов
def split_text_by_paragraphs(text, max_tokens=8000):
    paragraphs = text.split('\n')  # Разделение по параграфам
    new_nodes = []
    current_text = ""

    for para in paragraphs:
        if count_tokens(current_text + para) < max_tokens:
            current_text += para + "\n"
        else:
            # Если достигли лимита, сохраняем текущий текст как отдельный узел
            new_nodes.append(current_text.strip())
            current_text = para + "\n"

    # Добавляем последний кусок текста
    if current_text.strip():
        new_nodes.append(current_text.strip())

    return new_nodes

# Модифицированная функция создания нод
def create_nodes(documents):
    all_nodes = []
    for doc in documents:
        law_name = doc.metadata['file_name'].replace('.txt', '')
        law_text = doc.text
        nodes_list = re.split(r'\n(?=Статья \d+\.)', law_text.strip())
        
        for node_text in nodes_list:
            # Если количество токенов в ноде меньше 8000, добавляем её как есть
            if count_tokens(node_text) <= 8000:
                all_nodes.append(Document(text=node_text, metadata={'file_name': law_name}))
            else:
                # Если больше 8000, разбиваем на параграфы
                split_nodes = split_text_by_paragraphs(node_text)
                for split_node in split_nodes:
                    all_nodes.append(Document(text=split_node, metadata={'file_name': law_name}))

    return all_nodes

# Пример использования:
documents = load_documents(documents_file_path)
nodes = create_nodes(documents)

In [114]:
len(nodes)

4595

In [108]:
from llama_index.core import SimpleDirectoryReader, Document
import re


def load_documents(file_path):
    return SimpleDirectoryReader(file_path).load_data()

def create_nodes(documents):
    all_nodes = []
    for doc in documents:
        law_name = doc.metadata['file_name'].replace('.txt', '')
        law_text = doc.text
        nodes_list = re.split(r'\n(?=Статья \d+\.)', law_text.strip())
        nodes = [Document(text=t, metadata={'file_name': law_name}) for t in nodes_list]
        all_nodes.extend(nodes)

    return all_nodes


documents = load_documents(documents_file_path)
nodes = create_nodes(documents)
len(nodes)

4595

In [117]:
l = 0
i=0
for n in nodes:
    if len(str(n.text)) > l:
        l = len(n.text)
        i = nodes.index(n)

print(l)
print(nodes[i].text)

27132
Статья 174. Термины и определения, используемые в Особенной части настоящего Кодекса
В Особенной части настоящего Кодекса используются следующие термины и определения:
1) аффинированный мерный слиток - изготовленный и маркированный слиток из драгоценного металла (золота, серебра или платины) массой 1000 грамм и менее, с содержанием химически чистого основного металла не менее 99,90 процента лигатурной массы слитка для золота, не менее 99,90 процента лигатурной массы слитка для серебра и не менее 99,95 процента лигатурной массы слитка для платины;
2) аффинированный стандартный слиток - изготовленный и маркированный слиток из золота или серебра, соответствующий Международным стандартам качества, принятым Лондонской ассоциацией рынка драгоценных металлов;
3) безнадежный долг - сумма, причитающаяся налогоплательщику, которую налогоплательщик не в состоянии полностью получить вследствие прекращения обязательства по решению суда, банкротства, ликвидации или смерти должника, или истечен

In [101]:
import weaviate
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core import SimpleDirectoryReader, StorageContext, VectorStoreIndex
from llama_index.vector_stores.weaviate import WeaviateVectorStore


# def load_documents(file_path):
#     return SimpleDirectoryReader(file_path).load_data()


# def create_nodes(documents, chunk_size=chunk_size, chunk_overlap=chunk_overlap):
#     node_parser = SentenceSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
#     nodes = node_parser.get_nodes_from_documents(documents)
#     return nodes


def connect_index(weaviate_client):
    vector_store = WeaviateVectorStore(
        weaviate_client=weaviate_client,
        index_name=Org
    )

    storage_context = StorageContext.from_defaults(vector_store=vector_store)
    index = VectorStoreIndex([], storage_context=storage_context)
    return index


def insert_nodes_index(index, nodes):
    index.insert_nodes(nodes)


client = weaviate.connect_to_wcs(
            cluster_url=weaviate_url,
            auth_credentials=weaviate.AuthApiKey(api_key=API_KEY)
)
print("Weaviate client is ready:", client.is_ready())

documents = load_documents(documents_file_path)
nodes = create_nodes(split_documents)

index = connect_index(weaviate_client=client)
insert_nodes_index(index, nodes=nodes)

client.close()
print("Weaviate client connection closed.")

/home/codespace/.local/share/virtualenvs/legal-advisor-rag-x6CAXa7x/lib/python3.12/site-packages/weaviate/__init__.py:144: DeprecationWarning: Dep010: Importing AuthApiKey from weaviate is deprecated. Import AuthApiKey from its module: weaviate.auth
  _Warnings.root_module_import(name, map_[name])


Weaviate client is ready: True


/home/codespace/.local/share/virtualenvs/legal-advisor-rag-x6CAXa7x/lib/python3.12/site-packages/weaviate/warnings.py:305: ResourceWarning: Con004: The connection to Weaviate was not closed properly. This can lead to memory leaks.
            Please make sure to close the connection using `client.close()`.
  warnings.warn(


BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 8192 tokens, however you requested 12194 tokens (12194 in your prompt; 0 for the completion). Please reduce your prompt; or completion length.", 'type': 'invalid_request_error', 'param': None, 'code': None}}

# RAG flow

In [5]:
import weaviate
from llama_index.core import StorageContext, VectorStoreIndex
from llama_index.vector_stores.weaviate import WeaviateVectorStore

def connect_index(weaviate_client):
    vector_store = WeaviateVectorStore(
        weaviate_client=weaviate_client,
        index_name=Org
    )

    storage_context = StorageContext.from_defaults(vector_store=vector_store)
    index = VectorStoreIndex([], storage_context=storage_context)
    return index

weaviate_client = weaviate.connect_to_wcs(
    cluster_url=weaviate_url,
    auth_credentials=weaviate.AuthApiKey(api_key=API_KEY)
    )
print("Client is ready:", weaviate_client.is_ready())

index = connect_index(weaviate_client=weaviate_client)

/home/codespace/.local/share/virtualenvs/legal-advisor-rag-x6CAXa7x/lib/python3.12/site-packages/weaviate/__init__.py:144: DeprecationWarning: Dep010: Importing AuthApiKey from weaviate is deprecated. Import AuthApiKey from its module: weaviate.auth
  _Warnings.root_module_import(name, map_[name])


Client is ready: True


In [32]:
from llama_index.core.retrievers import VectorIndexRetriever

retriever = VectorIndexRetriever(
    index,
    vector_store_query_mode="hybrid",
    similarity_top_k = config['Retriever']['similarity_top_k'],
    alpha = 0.9, #config['Retriever']['alpha'],
    similarity_threshold = config['Retriever']['similarity_threshold'],
    )

def search_nodes(query):
    return retriever.retrieve(query)


def build_prompt(query, search_results):

    nodes_with_scores = [(node, node.score) for node in search_results]
    sorted_nodes = sorted(nodes_with_scores, key=lambda x: x[1], reverse=True)
    sorted_nodes_only = [node for node, score in sorted_nodes]

    context = ""
    
    for nod in sorted_nodes_only:
        context = context + nod.metadata['file_name'].replace('txt', '') + '\n' + nod.text + "\n\n"

    prompt_template = config['RAG_model']['prompt_template']
    prompt = prompt_template.format(language = language, question=query, context=context).strip()
    return prompt

def llm_response(prompt):
    response = llm_client.chat.completions.create(
        model=config['RAG_model']['model_name'],
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

def rag(query):
    search_results = search_nodes(query)
    prompt = build_prompt(query, search_results)
    answer = llm_response(prompt)
    return answer

In [33]:
query = "В каких случаях предусмотрена уголовная ответственность за подделку или уничтожение идентификационного номера транспортного средства, и какие виды наказания могут быть применены за совершение данного преступления?"
print(build_prompt(query, search_nodes(query)))

You're a legal advisor of Kyrgyz legislation. Without preamble, answer the QUESTION in English language based on the provided CONTEXT.
Use only the information from the CONTEXT when answering the QUESTION and not prior knowledge.
Provide short answers. Reffer in the answer to the name of the document from wich the context is extracted.

QUESTION:
В каких случаях предусмотрена уголовная ответственность за подделку или уничтожение идентификационного номера транспортного средства, и какие виды наказания могут быть применены за совершение данного преступления?

CONTEXT:
Уголовный кодекс Кыргызской Республики.
Статья 380. Подделка, уничтожение идентификационного номера транспортного средства
 
Подделка или уничтожение идентификационного номера, номера кузова, шасси, транспортного средства – 
наказываются штрафом от 500 до 1000 расчетных показателей либо лишением свободы на срок от двух до пяти лет.
 
Статья 381. Хищение документа, печати или штампа
 
1. Хищение официального документа, печат

In [13]:
answer = rag(query)
print(answer)

The context provided does not contain information regarding the procedure for the assignment of claims on order securities according to the legislation of the Kyrgyz Republic. Therefore, I cannot answer the question based on the documents referenced.
